In [1]:
import uuid
from uuid import UUID
import random
from collections import Counter
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import itertools

In [12]:
d = dict()
d['matthew'] = 'pizza'
d['jesse'] = 'cheese'
d['alycia'] = 'lasagna'
random.choice(list(d.keys()))
for key, value in d.items():
    print(key)

matthew
jesse
alycia


In [41]:
class GenomeSimulator:
    def __init__(self):
        self.generation = 1
        self.people = dict()
        self.start_year = 1800
    
    def setup(self, population=32):
        for _ in range(population):
            value = uuid.uuid4()
            self.people[value] = {'name':value,
                                  'genome': [value] * 20_000,
                                  'generation': self.generation,
                                  'sex':random.randint(0,1),
                                  'birth_year':random.randint(0,10) + self.start_year,
                                  'partner': None,
                                  'parents': None,
                                  'ancestors': []}
         
    def genome_similarity(self, person_a, person_b):
        matching = 0
        for gene in range(20_000):
            if person_a['genome'][gene] == person_b['genome'][gene]:
                matching += 1
        return matching / 20_000 * 100

    def make_partners(self):
        self.generation += 1
        candidates = {key:value for key, value in self.people.items() if value['partner'] is None}
        males = {key:value for key, value in candidates.items() if value['sex'] == 1}
        females = {key:value for key, value in candidates.items() if value['sex'] == 0}
        while len(males) > 0 and len(females) > 0:
            father = males[random.choice(list(males.keys()))]
            mother = females[random.choice(list(females.keys()))]
            similarity = self.genome_similarity(father,mother)
            if similarity < 2:
                for child in range(1,random.randint(1,8)):
                    genome = []
                    for gene in range(20_000):
                        if random.randint(0,1) == 0:
                            genome.append(father['genome'][gene])
                        else:
                            genome.append(mother['genome'][gene])
                    candidates[father['name']]['partner'] = mother['name']
                    candidates[mother['name']]['partner'] = father['name']
                    new_name = uuid.uuid4()
                    birth_year = max(father['birth_year'], mother['birth_year']) + random.randint(20,35)
                    ancestors = father['ancestors'] + [father['name']] + mother['ancestors'] + [mother['name']]
                    self.people[new_name] = {'name':new_name,
                                             'genome': genome,
                                             'generation': father['generation']+1,
                                             'sex': random.randint(0,1),
                                             'birth_year': birth_year,
                                             'partner': None,
                                             'parents':[father['name'],mother['name']],
                                             'ancestors':ancestors}
                males.pop(father['name'])
                females.pop(mother['name'])

In [42]:
def genome_similarity(person_a, person_b):
    matching = 0
    for gene in range(20_000):
        if person_a['genome'][gene] == person_b['genome'][gene]:
            matching += 1
    return matching / 20_000 * 100

In [43]:
simulation = GenomeSimulator()
simulation.setup(population=32)
len(simulation.people.items())

32

In [44]:
for generation in range(1,20):
    simulation.make_partners()
    print("generation %s: %s" % (generation, datetime.now()))

generation 1: 2023-05-25 17:15:39.959275
generation 2: 2023-05-25 17:15:41.206046
generation 3: 2023-05-25 17:15:42.768859


KeyboardInterrupt: 

In [ ]:
len(simulation.people)

In [ ]:
Counter([x['sex'] for x in simulation.people.values()])

In [ ]:
Counter([x['sex'] for x in simulation.people.values() if x['partner'] is None])

In [ ]:
simulation.generation

In [ ]:
list(simulation.people.values())[-1]

In [ ]:
l1 = []
count = 0
for item in simulation.people.values():
    if item['genome'] not in l1:
        count += 1
        l1.append(item['genome'])

In [ ]:
count

In [ ]:
len(simulation.people)

In [ ]:
unpartnered = pd.DataFrame(Counter([x['birth_year'] for x in simulation.people.values() if x['partner'] is None]).items()).sort_values(1)

In [ ]:
unpartnered.sort_values(0)

In [ ]:
plt.scatter(x=unpartnered[0], y=unpartnered[1])

In [ ]:
Counter([x['generation'] for x in simulation.people.values() if x['partner'] is None])

In [ ]:
datetime.now()

In [ ]:
Counter([x['generation'] for x in simulation.people.values()])

In [ ]:
last_generation = [x['name'] for x in simulation.people.values() if x['generation'] == 4]

In [ ]:
people = pd.DataFrame(simulation.people).transpose()
people

In [ ]:
fifth_gen = list(itertools.combinations(people[people['generation']==6].index,2))
relatedness = list()
for combo in fifth_gen:
    relatedness.append(genome_similarity(people.loc[combo[0]],
                                         people.loc[combo[1]]))
relatedness

In [ ]:
plt.hist(relatedness)

In [ ]:
people[people['generation']==6]

In [ ]:
fifth_gen = list(itertools.combinations(people[people['generation']==6].index,2))
for combo in fifth_gen:
    anc = set(people.loc[combo[0]]['ancestors'])
    if people.loc[combo[1]]['sex'] != people.loc[combo[0]]['sex']:
        print( len(anc & set(people.loc[combo[1]]['ancestors'])) / len(anc))